In [78]:
import pandas as pd
import datetime

def distribute_per_week(start, end, column_name, country):
    index = pd.date_range(start, end, freq="W").strftime('%Y-%U')
    df = index.to_frame(name='week')
    df['country'] = country
    df = df.set_index(['country','week'])
    df[column_name] = True
    return df 

def distribute_per_weeknumbers(row):
    """Gets a dataframe row and converts it into a time series dataframe binned in weeks"""
    date_end = row['date_end'] if row['date_end'] != 'NA' else datetime.date.today()
    df = distribute_per_week(row['date_start'], date_end, row['Response_measure'], row['Country'])
    return df

# The entry point function MUST have two input arguments.
# If the input port is not connected, the corresponding
# dataframe argument will be None.
#   Param<dataframe1>: a pandas.DataFrame
#   Param<dataframe2>: a pandas.DataFrame
def azureml_main(dataframe1 = None, dataframe2 = None):

    # Execution logic goes here

    measures_per_week = dataframe1.apply(distribute_per_weeknumbers, axis=1)

    # merge duplicate columns
    # transpose the initial dataframe '.T'
    # group by row which have now duplicate index values (which were column names)
    # apply the 'any' aggregation function 
    # transpose back the result
    merged_measures_per_week = pd \
    .concat(measures_per_week.array, axis=1) \
    .apply(lambda x: x.abs().fillna(0) if x.dtype.kind in 'iufc' else x.fillna(False)) \
    .T.groupby(level=0).any().T 

    merged_measures_per_week.index.set_names(['country', 'week'], inplace=True)

    deaths_per_week = dataframe2 \
    .drop(columns=['country_code','source']) \
    .rename(columns={'year_week':'week'}) \
    .set_index(['indicator','continent','country', 'week']) \
    .loc[('deaths','Europe')]

    deaths_per_week.index.set_names(['country', 'week'], inplace=True)

    result = deaths_per_week \
    .join(merged_measures_per_week) \
    .apply(lambda x: x.abs().fillna(0) if x.dtype.kind in 'iufc' else x.fillna(False)) \
    .sort_index()
    
    result=result.reset_index(level=['country', 'week'])
    result=result.drop(columns=['weekly_count','population','cumulative_count'])
    result.week = pd.to_datetime(result.week+'-0',format='%Y-%W-%w')

    result = result.set_index(['country','week'])
    result = result[~result.index.duplicated(keep='first')]
    result=result.reset_index(level=['country', 'week'])

    # If a zip file is connected to the third input port,
    # it is unzipped under "./Script Bundle". This directory is added
    # to sys.path. Therefore, if your zip file contains a Python file
    # mymodule.py you can import it using:
    # import mymodule
    return result

In [76]:
from azureml.core import Workspace, Dataset
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

dataset = Dataset.get_by_name(ws, name='covid-19-response-weekly')
dsf = dataset.to_pandas_dataframe()
dsf


capstone
ML
westeurope
f08c5f25-28be-4c21-993c-ad64d5c84d3a


,Country,Response_measure,date_start,date_end
0,Austria,AdaptationOfWorkplace,2020-03-10,NA
1,Austria,BanOnAllEvents,2020-11-03,NA
2,Austria,ClosDaycare,2020-03-16,2020-05-04
3,Austria,ClosDaycarePartial,2020-11-17,2020-12-07
4,Austria,ClosHigh,2020-03-16,2020-09-30
...,...,...,...,...
1430,United Kingdom,StayHomeRiskG,2020-03-16,2020-07-05
1431,United Kingdom,StayHomeRiskG,2020-11-05,2020-12-01
1432,United Kingdom,StayHomeRiskG,2021-01-05,NA
1433,United Kingdom,StayHomeRiskGPartial,2020-07-06,2020-08-01


In [71]:
dataset2 = Dataset.get_by_name(ws, name='covid-19-cases-deaths-weekly')
dsf2 = dataset2.to_pandas_dataframe()
dsf2



,country,country_code,continent,population,indicator,weekly_count,year_week,rate_14_day,cumulative_count,source
0,Afghanistan,AFG,Asia,38928341,cases,0,2020-01,NaN,0,"Epidemic intelligence, national weekly data"
1,Afghanistan,AFG,Asia,38928341,cases,0,2020-02,0.000000,0,"Epidemic intelligence, national weekly data"
2,Afghanistan,AFG,Asia,38928341,cases,0,2020-03,0.000000,0,"Epidemic intelligence, national weekly data"
3,Afghanistan,AFG,Asia,38928341,cases,0,2020-04,0.000000,0,"Epidemic intelligence, national weekly data"
4,Afghanistan,AFG,Asia,38928341,cases,0,2020-05,0.000000,0,"Epidemic intelligence, national weekly data"
...,...,...,...,...,...,...,...,...,...,...
21087,Zimbabwe,ZWE,Africa,14862927,deaths,34,2020-52,3.162230,354,"Epidemic intelligence, national weekly data"
21088,Zimbabwe,ZWE,Africa,14862927,deaths,26,2020-53,4.036890,380,"Epidemic intelligence, national weekly data"
21089,Zimbabwe,ZWE,Africa,14862927,deaths,127,2021-01,10.294069,507,"Epidemic intelligence, national weekly data"
21090,Zimbabwe,ZWE,Africa,14862927,deaths,206,2021-02,22.404739,713,"Epidemic intelligence, national weekly data"


In [77]:
res = azureml_main(dsf, dsf2)
res.to_csv('test.csv')
res

,country,week,rate_14_day,AdaptationOfWorkplace,AdaptationOfWorkplacePartial,BanOnAllEvents,BanOnAllEventsPartial,ClosDaycare,ClosDaycarePartial,ClosHigh,...,StayHomeGen,StayHomeGenPartial,StayHomeOrder,StayHomeOrderPartial,StayHomeRiskG,StayHomeRiskGPartial,Teleworking,TeleworkingPartial,WorkplaceClosures,WorkplaceClosuresPartial
0,Albania,2020-03-15,0.000000,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,Albania,2020-03-22,0.349354,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,Albania,2020-03-29,0.698708,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,Albania,2020-04-05,3.144185,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,Albania,2020-04-12,6.637724,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2885,the Holy See/ Vatican City State,2020-12-27,0.000000,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2886,the Holy See/ Vatican City State,2021-01-03,0.000000,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2887,the Holy See/ Vatican City State,2021-01-10,0.000000,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2888,the Holy See/ Vatican City State,2021-01-17,0.000000,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [51]:
res

,country,week,rate_14_day,AdaptationOfWorkplace,AdaptationOfWorkplacePartial,BanOnAllEvents,BanOnAllEventsPartial,ClosDaycare,ClosDaycarePartial,ClosHigh,...,StayHomeGen,StayHomeGenPartial,StayHomeOrder,StayHomeOrderPartial,StayHomeRiskG,StayHomeRiskGPartial,Teleworking,TeleworkingPartial,WorkplaceClosures,WorkplaceClosuresPartial
0,Albania,2020-03-09,0.000000,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,Albania,2020-03-16,0.349354,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,Albania,2020-03-23,0.698708,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,Albania,2020-03-30,3.144185,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,Albania,2020-04-06,6.637724,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2942,the Holy See/ Vatican City State,2020-12-28,0.000000,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2943,the Holy See/ Vatican City State,2021-01-04,0.000000,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2944,the Holy See/ Vatican City State,2021-01-04,0.000000,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2945,the Holy See/ Vatican City State,2021-01-11,0.000000,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [68]:
# i = res[(res.week == datetime.datetime(2020,1,4))].index
# res2 = res.drop(i)
# res2 = res.set_index(['country','week'])df.loc[df.index.drop_duplicates(keep='first')].groupby(level=0).first()
# res2
res1 = res.set_index(['country','week'])
# res2 = res2.loc[res2.index.drop_duplicates(keep='first')]
# res2 = res1.index.drop_duplicates(keep='first')
df3 = res1[~res1.index.duplicated(keep='first')]
# res2 = res1.loc[res2]
df3
# res2.to_csv('test.csv')

rate_14_day  \
country                          week                      
Albania                          2020-03-09     0.000000   
                                 2020-03-16     0.349354   
                                 2020-03-23     0.698708   
                                 2020-03-30     3.144185   
                                 2020-04-06     6.637724   
...                                                  ...   
the Holy See/ Vatican City State 2020-12-21     0.000000   
                                 2020-12-28     0.000000   
                                 2021-01-04     0.000000   
                                 2021-01-11     0.000000   
                                 2021-01-18     0.000000   

                                             AdaptationOfWorkplace  \
country                          week                                
Albania                          2020-03-09                  False   
                                 2020-03-16                  False   
                                 2020-03-23                  False   
                                 2020-03-30                  False   
                                 2020-04-06                  False   
...                                                            ...   
the Holy See/ Vatican City State 2020-12-21                  False   
                                 2020-12-28                  False   
                                 2021-01-04                  False   
                                 2021-01-11                  False   
                                 2021-01-18                  False   

                                             AdaptationOfWorkplacePartial  \
country                          week                                       
Albania                          2020-03-09                         False   
                                 2020-03-16                         False   
                                 2020-03-23                         False   
                                 2020-03-30                         False   
                                 2020-04-06                         False   
...                                                                   ...   
the Holy See/ Vatican City State 2020-12-21                         False   
                                 2020-12-28                         False   
                                 2021-01-04                         False   
                                 2021-01-11                         False   
                                 2021-01-18                         False   

                                             BanOnAllEvents  \
country                          week                         
Albania                          2020-03-09           False   
                                 2020-03-16           False   
                                 2020-03-23           False   
                                 2020-03-30           False   
                                 2020-04-06           False   
...                                                     ...   
the Holy See/ Vatican City State 2020-12-21           False   
                                 2020-12-28           False   
                                 2021-01-04           False   
                                 2021-01-11           False   
                                 2021-01-18           False   

                                             BanOnAllEventsPartial  \
country                          week                                
Albania                          2020-03-09                  False   
                                 2020-03-16                  False   
                                 2020-03-23                  False   
                                 2020-03-30                  False   
                                 2020-04-06                  False   
...                                      

In [53]:
res2

,country,week,rate_14_day,AdaptationOfWorkplace,AdaptationOfWorkplacePartial,BanOnAllEvents,BanOnAllEventsPartial,ClosDaycare,ClosDaycarePartial,ClosHigh,...,StayHomeGen,StayHomeGenPartial,StayHomeOrder,StayHomeOrderPartial,StayHomeRiskG,StayHomeRiskGPartial,Teleworking,TeleworkingPartial,WorkplaceClosures,WorkplaceClosuresPartial
0,Albania,2020-03-09,0.000000,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,Albania,2020-03-16,0.349354,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,Albania,2020-03-23,0.698708,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,Albania,2020-03-30,3.144185,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,Albania,2020-04-06,6.637724,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2942,the Holy See/ Vatican City State,2020-12-28,0.000000,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2943,the Holy See/ Vatican City State,2021-01-04,0.000000,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2944,the Holy See/ Vatican City State,2021-01-04,0.000000,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2945,the Holy See/ Vatican City State,2021-01-11,0.000000,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
